# BAOAB method in Julia

In [34]:
using LaTeXStrings, Random, Plots, Serialization, StatsBase, Base.Threads, LinearAlgebra

function A_step(qp, h)
    q, p = qp
    q = q + h * p
    return [q, p]
end

function B_step(qp, h, force)
    q, p = qp
    F = force
    p = p + h * F
    return [q, p]
end

function O_step(qp, h, A, beta)
    q, p = qp
    alpha = exp(-h * A)
    R = randn(length(q))
    p = alpha * p + sqrt(1 / beta) * sqrt(1 - exp(-2 * h * A)) * R
    return [q, p]
end

function BAOAB_step(q, p, h, A, beta, force)
    qp = copy([q, p])
    qp = B_step(qp, h/2, force)
    qp = A_step(qp, h/2)
    qp = O_step(qp, h, A, beta)
    qp = A_step(qp, h/2)
    qp = B_step(qp, h/2, force)
    q, p = qp
    return q, p
end

BAOAB_step (generic function with 1 method)

In [62]:
function grad_BLR(data, w, N, n)
    # 2nd model - Large Scale Bayesian Logistic Regression
    dim = size(data, 2) - 1
    x = data[:, 1:dim]
    y = data[:, end]

    sum = zeros(dim)
    for i in 1:n
        a = exp(- y[i] * dot(w, x[i, :]))
        sum = sum .+ ((-y[i] * a  / (1 + a)) .* x[i, :])
    end

    w += sum .* N/n

    return w
end

function run_simulation(q0, p0, Nsteps, h, A, beta, Samples, step_function, grad_U, N, n)
    dim = size(Samples, 2) - 1
    q_traj = zeros(dim,Nsteps)
    p_traj = zeros(dim,Nsteps)
    t_traj = zeros(Nsteps)

    q = copy(q0)
    p = copy(p0)
    t = 0.0

    for i in 1:Nsteps
        idx = randperm(N)[1:n]
        data = Samples[idx,:]
        force = - grad_U(data, q, N, n)
        q, p = step_function(q, p, h, A, beta, force)
        t += h
        
        q_traj[:,i] = q
        p_traj[:,i] = p
        t_traj[i] = t
    end

    return q_traj, p_traj, t_traj
end

function sigmoid(z)
    return 1.0 ./ (1.0 .+ exp.(-z))
end

function predict_BLR(features, weights)
    # Calculate the linear combination of features and weights
    z = first(features' * weights)

    # Apply the logistic function to get the probability
    probabilities = sigmoid(z)

    # Apply threshold (0.5) for binary classification
    predictions = ifelse.(probabilities .>= 0.5, 1, -1)
    
    return predictions
end


predict_BLR (generic function with 1 method)

In [29]:
function save_variable(variable, file_name)
    name = string(file_name, ".jls")
    open(name, "w") do file
        serialize(file, variable)
    end
end

save_variable (generic function with 1 method)

In [31]:
using MLDatasets

train_x, train_y = MNIST(split=:train)[:]
ind = findall(x -> x == 7 || x == 9, train_y)
y_train = train_y[ind]
y_train = ifelse.(y_train .== 7, 1, -1)
x_train = transpose(reshape(train_x, size(train_x, 1)*size(train_x, 2), size(train_x, 3))[:, ind])
train = hcat(x_train, y_train)

test_x, test_y = MNIST(split=:test)[:]
ind = findall(x -> x == 7 || x == 9, test_y)
y_test = test_y[ind]
y_test = ifelse.(y_test .== 7, 1, -1)
x_test = transpose(reshape(test_x, size(test_x, 1)*size(test_x, 2), size(test_x, 3))[:, ind]);

In [35]:
# Initialize one walker
dim = size(train, 2) - 1
q0 = randn(dim)   # just for initialization
p0 = randn(dim)

Nsteps = 10000
h = 0.001
A = 1.0
beta = 1.0
N = size(train, 1)
n = 100

# Run one long trajectory of Nsteps, using the BAOAB scheme
q_traj, p_traj, t_traj = run_simulation(q0, p0, Nsteps, h, A, beta, train, BAOAB_step, grad_BLR, N, n);

In [69]:
w_mean = mean(q_traj, dims=2)
predicted_labels = zeros(size(x_test, 1))
for i in 1:size(x_test, 1)
    predicted_labels[i] = predict_BLR(x_test[i,:], w_mean)
end
accuracy = sum(predicted_labels .== y_test) / length(y_test)

0.9646539027982327